# Scratch Artificial Neural Network

In [1]:
from Layer import InputLayer
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.dont_write_bytecode = True

# Import libraries
import torch
import numpy as np
import pandas as pd


from ArtificialNeuralNetwork import ArtificialNeuralNetwork
from Layer import Layer, InputLayer, OutputLayer
from Function import ActivationFunction, LossFunction
from enums import InitializerType

In [3]:
# Disable __grad based on assistant direction
torch.set_grad_enabled(False)

# Pipeline for testing MNIST dataset

In [4]:
# Const variables
input_size = 784
hidden_layers = 1
hidden_size = 2
output_size = 10
learning_rate = 0.1
param_1 = 0
param_2 = 0.5

In [5]:
# Import Dataset
train = pd.read_csv("data/train.csv")

In [6]:
# Data Preprocessing
data = np.array(train)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape
X_train = X_train.T

X_dev = X_dev.T

In [7]:
# Initialize Artificial Neural Network
ann = ArtificialNeuralNetwork(
    123,
    LossFunction.mean_squared_error,
    InputLayer(
        input_size=input_size,
        layer_name="Input"
    ),
    *[Layer(
        weight_init=InitializerType.RANDOM_DIST_NORMAL,
        bias_init=InitializerType.RANDOM_DIST_NORMAL,
        input_size=input_size,
        output_size=hidden_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.relu,
        layer_name=f"Hidden Layer {i}"
    ) for i in range(hidden_layers)],
    OutputLayer(
        weight_init=InitializerType.RANDOM_DIST_NORMAL,
        bias_init=InitializerType.RANDOM_DIST_NORMAL,
        input_size=hidden_size,
        output_size=output_size,
        param_1=param_1,
        param_2=param_2,
        activation=ActivationFunction.softmax,
        layer_name="Output Layer"
    )
)

In [ ]:
# Train Artificial Neural Network
ann.train(
    torch.tensor(X_train),
    torch.tensor(Y_train),
    learning_rate,
    500,
    LossFunction.mean_squared_error,
    verbose = True
)

Epoch 0 - Loss: 25.054248809814453
Epoch 1 - Loss: 12.527289390563965
Epoch 2 - Loss: 8.351560592651367
Epoch 3 - Loss: 6.263682842254639
Epoch 4 - Loss: 5.010952472686768
Epoch 5 - Loss: 4.175796985626221
Epoch 6 - Loss: 3.579256772994995


In [ ]:
# Test Artificial Neural Network
ann.test(torch.tensor(X_dev), torch.tensor(Y_dev), LossFunction.mean_squared_error)